# Bring your own tools to the enterprise gateway
In this notebook, you'll augment the existing AgentCore Gateway with your own tools of choice. This is your chance to experiment and publish reusable tools that could be made available to others via the gateway. 
- Lambda function targets
- OpenAPI schema targets
- Smithy model targets
- MCP servers targets
- Built-in templates from integration providers as targets

You can find more information for each of these target options  here: https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway-supported-targets.html 

![List](images/gateway-targets.png)

## 1. Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude Sonnet 4 model on Amazon Bedrock
- Basic understanding of Python programming

In [ ]:
%pip install -qU boto3 strands-agents strands-agents-tools defusedxml httpx bedrock_agentcore_starter_toolkit

## 2. Lambda targets: 

In the previous lab, we used the database tools from Biomni as a lambda function target for the AgentCore Gateway. You could explore reusing other tool categories from Biomni as available [here](https://github.com/snap-stanford/Biomni/tree/main/biomni/tool) . 

For example, we demonstrate literature tools from Biomni as a second target to the gateway  in the [AWS Life sciences sample agents repository](https://github.com/aws-samples/amazon-bedrock-agents-healthcare-lifesciences/tree/main/agents_catalog/28-Research-agent-biomni-gateway-tools/prerequisite/lambda-literature). 

You can add the tools to your existing Gateway as another lambda target with the help of this [notebook demonstration](https://github.com/awslabs/amazon-bedrock-agentcore-samples/blob/main/01-tutorials/02-AgentCore-gateway/01-transform-lambda-into-mcp-tools/01-gateway-target-lambda-oauth.ipynb)


## 3. MCP server as a target

Another option is to bring a MCP server as a target. AgentCore Gateway now supports onboarding pre-existing MCP server implementations as targets, in addition to Lambda and OpenAPI tools. 

Create the MCP server target for the gateway. We use the [AWS Knowledge MCP Server](https://awslabs.github.io/mcp/servers/aws-knowledge-mcp-server) : A fully managed remote MCP server that provides up-to-date documentation, code samples, knowledge about the regional availability of AWS APIs and CloudFormation resources, and other official AWS content.

In [ ]:
# Try with AWS Knowledge MCP Server
# https://awslabs.github.io/mcp/servers/aws-knowledge-mcp-server

import boto3
from utils import get_ssm_parameter
gatewayID = get_ssm_parameter("/deep-research-workshop/agentcore/gateway_id")
# Get gateway endpoint
gateway_endpoint = get_ssm_parameter("/deep-research-workshop/agentcore/gateway_url")
print(f"Gateway Endpoint - MCP URL: {gateway_endpoint}")
gateway_client = boto3.client("bedrock-agentcore-control")

# Create the Gateway Target for the AWS Knowledge MCP Server
create_gateway_target_response = gateway_client.create_gateway_target(
    name="aws-knowledge-server-target",
    gatewayIdentifier=gatewayID,
    targetConfiguration={
        "mcp": {"mcpServer": {"endpoint": "https://knowledge-mcp.global.api.aws"}}
    },
)
print(create_gateway_target_response)

In [ ]:
# Check that the Gateway target exists, and is READY
list_targets_response = gateway_client.list_gateway_targets(gatewayIdentifier=gatewayID)
print(list_targets_response)

### Check if the new MCP Server tools are now available through the Gateway

In [ ]:
from database_tools import (
    get_gateway_access_token,
    get_all_mcp_tools_from_mcp_client,
    tool_search,
    tools_to_strands_mcp_tools,
)
from utils import get_ssm_parameter
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp import MCPClient
import time

# Get gateway access token
jwt_token = get_gateway_access_token()
if not jwt_token:
    print("❌ Failed to get gateway access token")

# Get gateway endpoint
gateway_endpoint = get_ssm_parameter("/deep-research-workshop/agentcore/gateway_url")
print(f"Gateway Endpoint - MCP URL: {gateway_endpoint}")

# Create MCP client
client = MCPClient(
    lambda: streamablehttp_client(
        gateway_endpoint, headers={"Authorization": f"Bearer {jwt_token}"}
    )
)

with client:
    print("📋 Getting all available tools...")
    start_time = time.time()
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    list_time = time.time() - start_time
    print(f"✅ Found {len(all_tools)} total tools in {list_time:.2f}s\n")
    for tool in all_tools:
        print(tool.tool_name)

## 4 Augment your agent to access the new MCP server tools via the Gateway

You now have a gateway that combines tools for life sciences research with AWS infrastructure and knowledge tools. We can now use it to showcase the power of combining AWS infrastructure knowledge with domain-specific research data and literature.

You can test out the following example prompts : 
 
    "What AWS services are available in eu-west-1 for healthcare data processing?",  
    "I need to store 5TB of medical imaging data (DICOM files) that will be accessed 
    frequently for the first 3 months, then rarely after that. What's the most 
    cost-effective AWS storage solution?",  
    "We're launching a clinical trial in Germany. Is Amazon Bedrock available in 
    eu-central-1, and what models can we use there?",  
   

In [ ]:
from utils import get_ssm_parameter
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands.models import BedrockModel
import time

In [ ]:
QUERY = "What AWS services are available in eu-west-1 for healthcare data processing?"
MAX_TOOLS = 5
# Create Bedrock model
model = BedrockModel(
        model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
        temperature=0.7,
        streaming=True,
)
with client:
    # Use semantic tool search
    search_query_to_use = QUERY
    print(f"\n🔍 Searching for tools with query: '{search_query_to_use}'")

    start_time = time.time()
    tools_found = tool_search(
        gateway_endpoint, jwt_token, search_query_to_use, max_tools=MAX_TOOLS
    )
    search_time = time.time() - start_time

    if not tools_found:
        print("❌ No tools found from search")

    print(f"✅ Found {len(tools_found)} relevant tools in {search_time:.2f}s")
    print(f"Top tool: {tools_found[0]['name']}")


    agent_tools = tools_to_strands_mcp_tools(tools_found, MAX_TOOLS, client)
    agent = Agent(model=model, tools=agent_tools)
    response = agent(QUERY)
    print(response)

Deploy to Amazon Bedrock AgentCore Runtime

In [ ]:
SYSTEM_PROMPT = """You are a Healthcare Research Infrastructure Assistant specializing in AWS-powered life sciences solutions.

CAPABILITIES:
- AWS Infrastructure: Access AWS documentation, regional availability, service recommendations, and CloudFormation templates via AWS Knowledge tools
- Research Data: Query clinical trials database, patient outcomes, biomarker studies, and proprietary research data via database tools

APPROACH:
1. Understand the research or infrastructure need
2. Use AWS Knowledge tools for architecture, services, compliance, and regional availability
3. Use database tools to validate with real research data or inform design decisions
4. Combine both sources to provide data-driven, compliant AWS solutions

GUIDELINES:
- Prioritize HIPAA/GDPR compliance for healthcare data
- Consider cost optimization for large-scale research workloads
- Validate architectural decisions with actual research data when available
- Provide specific AWS service recommendations with regional availability
- Include relevant code samples or CloudFormation snippets when helpful

Be concise, technical, and always ground recommendations in both AWS best practices and real research context."""


In this step, we'll deploy the agent definition found in the `agent` folder to Amazon Bedrock AgentCore runtime. Let's start by taking a look at our agent code. Notice the `@app.entrypoint` decorator on the `strands_agent_bedrock` function. This is how we tell the AgentCore Runtime how to run the agent.

In [ ]:
%pycat agent.py

In [ ]:
import boto3
from bedrock_agentcore_starter_toolkit import Runtime

ssm = boto3.client("ssm")

agentcore_runtime = Runtime()
agentcore_runtime.configure(
    agent_name="researchapp_byo_mcpserver",
    auto_create_ecr=True,
    execution_role=ssm.get_parameter(
        Name="/deep-research-workshop/agentcore-runtime-tools-role-arn"
    )["Parameter"]["Value"],
    entrypoint="agent.py",
    requirements_file="requirements.txt",
)

In [ ]:
agentcore_runtime.launch(auto_update_on_conflict=True)

In [ ]:
import uuid
session_id = str(uuid.uuid4())
from invoke_agentcore import invoke_agentcore

invoke_agentcore(
    agent_runtime_name="researchapp_byo_mcpserver",
    prompt="what can you do?",
    session_id=session_id,
)

## 5.  Interact with the agent with a Streamlit UI

Follow these steps to open an interactive chat session with your new agent. See the 'Getting started section' of the workshop studio page to access a Streamlit chat application. 

1. Launch the deployed Streamlit UI application
2. Use the user credentials you created to log in
3. Select the region 'us-west-2' and agent `researchapp_byo_mcpserver` 
4. Ask your question in the chatbox - see reference question below.




Below, we provide an example query that combines both AWS infrastructure and Life sciences research. 

In [ ]:
QUERY = """ Create a complete architecture for real-time cancer biomarker discovery that:
- Uses AWS services available in this region
- Integrates with clinical trials database
- Incorporates latest PubMed research
- Meets all regulatory requirements""" 


## 6. (Optional) Clean Up

Run the next notebook cell to delete the AgentCore runtime environment.

In [ ]:
import boto3

agentcore_client = boto3.client("bedrock-agentcore-control")
agent_status = agentcore_runtime.status()

agentcore_client.delete_agent_runtime(agentRuntimeId=agent_status.config.agent_id)

In [ ]:
gatewayId = get_ssm_parameter("/deep-research-workshop/agentcore/gateway_id")
print("Deleting all targets for gateway", gatewayId)
list_response = gateway_client.list_gateway_targets(
            gatewayIdentifier = gatewayId,
            maxResults=100
)
for item in list_response['items']:
    targetId = item["targetId"]
    print("Deleting target ", targetId)
    gateway_client.delete_gateway_target(
            gatewayIdentifier = gatewayId,
            targetId = targetId
    )
    time.sleep(5)
print("Deleting gateway ", gatewayId)
gateway_client.delete_gateway(gatewayIdentifier = gatewayId)
    